# <font color=red>HW02-q2 Vision Course, Camera Calibration</font>
This is the notebook for **q2.py** that is the implementation of **Camera Calibration**. <br>
The code is written by **Asal Mehradfar** with student number **96105434**.

## <font color=orange>Description</font>

*   In the course we had: <br>
$$K =\begin{bmatrix} f_x & 0 & P_x \\ 0 & f_y & P_y \\ 0 & 0 & 1 \end{bmatrix}$$ <br>
*   For the changeable center we have four matrices for K that by comparing errors we find the matrix below:<br>
$$K =\begin{bmatrix} 2932 & 0 & 912 \\ 0 & 2953 & 552 \\ 0 & 0 & 1 \end{bmatrix}$$ <br>
Pay attention that the matrices which were found are almost the same in four cases.
*   For the non-changeable center we have four matrices for K that by comparing errors we find the matrix below:<br>
$$K =\begin{bmatrix} 2916 & 0 & 750 \\ 0 & 2934 & 500 \\ 0 & 0 & 1 \end{bmatrix}$$ <br>
Pay attention that the matrices which were found are almost the same in four cases.
*   Totally we can say that the result for first ten images is more accurate, but
all the results were almost the same.



### <font color=yellow>Imports</font>

In [79]:
import cv2
import numpy as np

### <font color=yellow>Parameters</font>

*   BOARD_SIZE = (6, 9)
*   a = 22 mm

In [80]:
BOARD_SIZE = (6, 9)
a = 22

### <font color=yellow>Functions</font>

In [81]:
def get_gray_img(path):
    """
    Read the image file from the path and change it to the gray one

    Inputs:
    --> path: path for the image
    Outputs:
    ==> gray_img: the gray image
    """
    img = cv2.imread(path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img

def calibration_camera(start, end, flags= 0):
    """
    for this implementation I used the code for the opencv source of camera calibration,
    we have two important parameters object_points and image_points,
    object_points are the grid ones which was made by np.mgrid, showing the boxes
    with their depth to be zero,
    img_points are showing the corners which was made by np.findChessboardCorners.
    we give this inputs to cv2.calibrateCamera and get the K matrix at the end.
    the other parameters and this matrix are used to find out whether our approximation
    works well or not.

    Inputs:
    --> start: num of the first image for camera calibration
    --> end: num of the last image for camera calibration
    --> flags: if zero the principal point is in the center of the table,
    if cv2.CALIB_FIX_PRINCIPAL_POINT the principal point is the center of the whole image
    Outputs:
    ==> err: the re-projection error, which gives a good estimation of
    how exact the found parameters are
    ==> mat: the parameters matrix, we named it K in out course
    """
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    object_points = []
    img_points = []

    obj_point = np.zeros((BOARD_SIZE[0] * BOARD_SIZE[1], 3), np.float32)
    obj_point[:,:2] = np.mgrid[0:BOARD_SIZE[0],0:BOARD_SIZE[1]].T.reshape(-1,2)
    obj_point = obj_point * 22 # multiply a to obj_points

    for i in range(start, end+1):
        img = get_gray_img('im' + str(i).zfill(2) + '.jpg')
        ret, corners = cv2.findChessboardCorners(img, BOARD_SIZE,
                        cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if ret:
            object_points.append(obj_point)
            # not necessary
            corners2 = cv2.cornerSubPix(img, corners, (11,11), (-1,-1), criteria)
            img_points.append(corners2)

    _,mat, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, img_points, img.shape[::-1], None, None, flags=flags)
    mean_error = 0
    for i in range(len(object_points)):
        img_points2, _ = cv2.projectPoints(object_points[i], rvecs[i], tvecs[i], mat, dist)
        error = cv2.norm(img_points[i], img_points2, cv2.NORM_L2) / len(img_points2)
        mean_error += error
    err = mean_error/len(object_points)

    return err, mat

### <font color=yellow>Main Part</font>

#### <font color=yellow>With the center of the chessboard as a principal point</font>

In [82]:
err1, m1 = calibration_camera(1, 10)
err2, m2 = calibration_camera(6, 15)
err3, m3 = calibration_camera(11, 20)
err4, m4 = calibration_camera(1, 20)

In [83]:
print(err1, '\n', m1)

0.04784290310559341 
 [[2.93177482e+03 0.00000000e+00 9.11515612e+02]
 [0.00000000e+00 2.95269125e+03 5.51569282e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [84]:
print(err2, '\n', m2)

0.05520849690235474 
 [[3.00205134e+03 0.00000000e+00 8.81603273e+02]
 [0.00000000e+00 2.99789505e+03 5.28410868e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [85]:
print(err3, '\n', m3)

0.05707281575713048 
 [[3.04620462e+03 0.00000000e+00 7.18349258e+02]
 [0.00000000e+00 3.03488553e+03 5.48592497e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [86]:
print(err4, '\n', m4)


0.05598006332469104 
 [[2.98249080e+03 0.00000000e+00 8.36993933e+02]
 [0.00000000e+00 2.99047669e+03 5.12819189e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


#### <font color=yellow>With the center of the image as a principal point</font>

In [87]:
err1, m1 = calibration_camera(1, 10, cv2.CALIB_FIX_PRINCIPAL_POINT)
err2, m2 = calibration_camera(6, 15, cv2.CALIB_FIX_PRINCIPAL_POINT)
err3, m3 = calibration_camera(11, 20, cv2.CALIB_FIX_PRINCIPAL_POINT)
err4, m4 = calibration_camera(1, 20, cv2.CALIB_FIX_PRINCIPAL_POINT)

In [88]:
print(err1, '\n', m1)

0.04940488249431769 
 [[2.91551292e+03 0.00000000e+00 7.49500000e+02]
 [0.00000000e+00 2.93403525e+03 4.99500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [89]:
print(err2, '\n', m2)

0.05627709471433159 
 [[3.01211525e+03 0.00000000e+00 7.49500000e+02]
 [0.00000000e+00 3.00442525e+03 4.99500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [90]:
print(err3, '\n', m3)

0.057434394320965435 
 [[3.03420019e+03 0.00000000e+00 7.49500000e+02]
 [0.00000000e+00 3.02427348e+03 4.99500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [91]:
print(err4, '\n', m4)

0.05637766018301391 
 [[2.98860107e+03 0.00000000e+00 7.49500000e+02]
 [0.00000000e+00 2.99499501e+03 4.99500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
